In [11]:
import numpy as np
from scipy.stats import chi2

In [12]:
import math
from numba import njit, jit

# https://github.com/etal/biofrills/blob/36684bb6c7632f96215e8b2b4ebc86640f331bcd/biofrills/stats/chisq.py
MACHEP = 1e-13  # the machine roundoff error / tolerance
BIG = 4e15
BIGINV =  1/BIG

@njit
def chisqprob_our(x, df):
    """
    Probability value (1-tail) for the Chi^2 probability distribution.

    Broadcasting rules apply.

    Parameters
    ----------
    x : array_like or float > 0
    df : array_like or float, probably int >= 1

    Returns
    -------
    chisqprob : ndarray
        The area from `chisq` to infinity under the Chi^2 probability
        distribution with degrees of freedom `df`.

    """
    if x <= 0:
        return 1.0
    if x == 0:
        return 0.0
    if df <= 0:
        raise ValueError("Domain error.")
    if x < 1.0 or x < df:
        return 1.0 - _igam(0.5*df, 0.5*x)
    return _igamc(0.5*df, 0.5*x)

@njit
def _igamc(a, x):
    """
    In this implementation both arguments must be positive.
    The integral is evaluated by either a power series or
    continued fraction expansion, depending on the relative
    values of a and x.
    """
    # Compute  x**a * exp(-x) / Gamma(a)
    ax = math.exp(a * math.log(x) - x - math.lgamma(a))

    # Continued fraction
    y = 1.0 - a
    z = x + y + 1.0
    c = 0.0
    pkm2 = 1.0
    qkm2 = x
    pkm1 = x + 1.0
    qkm1 = z * x
    ans = pkm1 / qkm1
    while True:
        c += 1.0
        y += 1.0
        z += 2.0
        yc = y * c
        pk = pkm1 * z - pkm2 * yc
        qk = qkm1 * z - qkm2 * yc
        if qk != 0:
            r = pk/qk
            t = abs((ans - r) / r)
            ans = r
        else:
            t = 1.0
        pkm2 = pkm1
        pkm1 = pk
        qkm2 = qkm1
        qkm1 = qk
        if abs(pk) > BIG:
                pkm2 *= BIGINV
                pkm1 *= BIGINV
                qkm2 *= BIGINV
                qkm1 *= BIGINV
        if t <= MACHEP:
            return ans * ax

@njit
def _igam(a, x):
    """
    Left tail of incomplete Gamma function
    """
    # Compute  x**a * exp(-x) / Gamma(a)
    ax = math.exp(a * math.log(x) - x - math.lgamma(a))

    # Power series
    r = a
    c = 1.0
    ans = 1.0

    while True:
        r += 1.0
        c *= x/r
        ans += c
        if c / ans <= MACHEP:
            return ans * ax / a

In [13]:
polyval([1, 2, 3], 2)

C:\Users\vasiliev\AppData\Roaming\Python\Python310\site-packages\numba\core\ir_utils.py:2172: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'p' of function 'polyval'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_27488\1153704916.py", line 8:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


11

In [18]:
SCIPY_EULER = 0.577215664901532860606512090082402431
MAXLOG = 709.79  # 709.782712893383973096206318587
lanczos_g = 6.024680040776729583740234375

igam_MAXITER = 2000
lgamma_05 = 0.5723649429247004

@njit
def polyval(p : np.ndarray, x):
    y = 0
    for pv in p:
        y = y * x + pv
    return y

@njit
def didonato_SN(a, x, N, tolerance):
    '''
    Computation of the Incomplete Gamma Function Ratios and their Inverse
    ARMIDO R. DIDONATO and ALFRED H. MORRIS, JR.
    ACM Transactions on Mathematical Software, Vol. 12, No. 4,
    December 1986, Pages 377-393.
    See equation 34.
    '''
    _sum = 1.0

    if (N >= 1):
        partial = x / (a + 1)

        _sum += partial
        for i in range (2, N+1):
            partial *= x / (a + i)
            _sum += partial
            if(partial < tolerance):
                break
    return _sum

@njit
def find_inverse_s(p, q):
    '''
    Computation of the Incomplete Gamma Function Ratios and their Inverse
    ARMIDO R. DIDONATO and ALFRED H. MORRIS, JR.
    ACM Transactions on Mathematical Software, Vol. 12, No. 4,
    December 1986, Pages 377-393.
    See equation 32.
    '''
    a = [0.213623493715853, 4.28342155967104, 11.6616720288968, 3.31125922108741]
    b = [0.3611708101884203e-1, 1.27364489782223, 6.40691597760039, 6.61053765625462, 1]

    if (p < 0.5):
        t = np.sqrt(-2 * np.log(p))
    else:
        t = np.sqrt(-2 * np.log(q))
    s = t - polyval(a, t) / polyval(b, t)
    if (p < 0.5):
        s = -s
    return s

@njit
def ratevl(x, num, denom):  # N = M = 12
    absx = np.abs(x);
    if (absx > 1):
        '''Evaluate as a polynomial in 1/x.'''
        num_ans = polyval(num[::-1], 1 / x)
        denom_ans = polyval(denom[::-1], 1 / x)
        return np.power(x, 0) * num_ans / denom_ans;
    else:
        num_ans = polyval(num, x)
        denom_ans = polyval(denom, x)
    return num_ans / denom_ans

@njit
def lanczos_sum_expg_scaled(x):
    lanczos_sum_expg_scaled_num = np.array([
        0.00606184234, 0.50984166556, 19.5199278824, 449.944556906, 
        6955.99960251, 75999.2930401, 601859.617168, 3481712.15498, 
        14605578.0876, 43338889.3246, 86363131.2881, 103794043.116, 56906521.9134
    ])

    lanczos_sum_expg_scaled_denom = np.array([
        1, 66, 1925, 32670, 357423, 2637558, 13339535,
        45995730, 105258076, 150917976, 120543840, 39916800, 0
    ])
    return ratevl(x, lanczos_sum_expg_scaled_num, lanczos_sum_expg_scaled_denom)

@njit
def igam_fac(a, x):
    if (np.abs(a - x) > 0.4 * np.abs(a)):
        ax = a * np.log(x) - x - lgamma_05
        if (ax < -MAXLOG):
            return 0.0
        return np.exp(ax)

    fac = a + lanczos_g - 0.5
    res = np.sqrt(fac / np.exp(1)) / lanczos_sum_expg_scaled(a)

    if ((a < 200) and (x < 200)):
        res *= np.exp(a - x) * pow(x / fac, a)
    else:
        num = x - a - lanczos_g + 0.5
        res *= np.exp(a * (np.log1p(num / fac) - num / fac) + x * (0.5 - lanczos_g) / fac)
    return res

def find_inverse_gamma(a, p, q):
    '''
      In order to understand what's going on here, you will
      need to refer to:

      Computation of the Incomplete Gamma Function Ratios and their Inverse
      ARMIDO R. DIDONATO and ALFRED H. MORRIS, JR.
      ACM Transactions on Mathematical Software, Vol. 12, No. 4,
      December 1986, Pages 377-393.
    '''
    if (a == 1):
        if (q > 0.9):
            result = -np.log1p(-p)
        else:
            result = -np.log(q)
    elif (a < 1):
        g = math.gamma(a)
        b = q * g

        if ((b > 0.6) or ((b >= 0.45) and (a >= 0.3))):
            '''
            DiDonato & Morris Eq 21:
            
            There is a slight variation from DiDonato and Morris here:
            the first form given here is unstable when p is close to 1,
            making it impossible to compute the inverse of Q(a,x) for small
            q. Fortunately the second form works perfectly well in this case.
            '''
            if ((b * q > 1e-8) and (q > 1e-5)):
                u = np.power(p * g * a, 1 / a)
            else:
                u = np.exp((-q / a) - SCIPY_EULER)
            result = u / (1 - (u / (a + 1)))
        elif ((a < 0.3) and (b >= 0.35)):
            '''DiDonato & Morris Eq 22:'''
            t = np.exp(-SCIPY_EULER - b)
            u = t * np.exp(t)
            result = t * np.exp(u)
        elif ((b > 0.15) or (a >= 0.3)):
            '''DiDonato & Morris Eq 23:'''
            y = -np.log(b)
            u = y - (1 - a) * np.log(y)
            result = y - (1 - a) * np.log(u) - np.log(1 + (1 - a) / (1 + u))
        elif (b > 0.1):
            '''DiDonato & Morris Eq 24:'''
            y = -np.log(b)
            u = y - (1 - a) * np.log(y)
            result = y - (1 - a) * np.log(u) - np.log((u * u + 2 * (3 - a) * u + (2 - a) * (3 - a)) / (u * u + (5 - a) * u + 2))
        else:
            '''DiDonato & Morris Eq 25:'''
            y = -np.log(b)
            c1 = (a - 1) * np.log(y)
            c1_2 = c1 * c1
            c1_3 = c1_2 * c1
            c1_4 = c1_2 * c1_2
            a_2 = a * a
            a_3 = a_2 * a

            c2 = (a - 1) * (1 + c1)
            c3 = (a - 1) * (-(c1_2 / 2) + (a - 2) * c1 + (3 * a - 5) / 2)
            c4 = (a - 1) * ((c1_3 / 3) - (3 * a - 5) * c1_2 / 2 + (a_2 - 6 * a + 7) * c1 +
                                   (11 * a_2 - 46 * a + 47) / 6)
            c5 = (a - 1) * (-(c1_4 / 4) + (11 * a - 17) * c1_3 / 6 + (-3 * a_2 + 13 * a - 13) * c1_2 +
                                   (2 * a_3 - 25 * a_2 + 72 * a - 61) * c1 / 2 +
                                   (25 * a_3 - 195 * a_2 + 477 * a - 379) / 12)

            y_2 = y * y
            y_3 = y_2 * y
            y_4 = y_2 * y_2
            result = y + c1 + (c2 / y) + (c3 / y_2) + (c4 / y_3) + (c5 / y_4)
    else:
        '''DiDonato and Morris Eq 31:'''
        s = find_inverse_s(p, q)

        s_2 = s * s
        s_3 = s_2 * s
        s_4 = s_2 * s_2
        s_5 = s_4 * s
        ra = np.sqrt(a)

        w = a + s * ra + (s_2 - 1) / 3
        w += (s_3 - 7 * s) / (36 * ra)
        w -= (3 * s_4 + 7 * s_2 - 16) / (810 * a)
        w += (9 * s_5 + 256 * s_3 - 433 * s) / (38880 * a * ra)

        if ((a >= 500) and (abs(1 - w / a) < 1e-6)):
            result = w
        elif (p > 0.5):
            if (w < 3 * a):
                result = w
            else:
                D = np.max(2, a * (a - 1))
                lg = lgamma_05
                lb = np.log(q) + lg
                if (lb < -D * 2.3):
                    '''DiDonato and Morris Eq 25:'''
                    y = -lb
                    c1 = (a - 1) * np.log(y)
                    c1_2 = c1 * c1
                    c1_3 = c1_2 * c1
                    c1_4 = c1_2 * c1_2
                    a_2 = a * a
                    a_3 = a_2 * a

                    c2 = (a - 1) * (1 + c1)
                    c3 = (a - 1) * (-(c1_2 / 2) + (a - 2) * c1 + (3 * a - 5) / 2)
                    c4 = (a - 1) * ((c1_3 / 3) - (3 * a - 5) * c1_2 / 2 + (a_2 - 6 * a + 7) * c1 +
                                           (11 * a_2 - 46 * a + 47) / 6)
                    c5 = (a - 1) * (-(c1_4 / 4) + (11 * a - 17) * c1_3 / 6 + (-3 * a_2 + 13 * a - 13) * c1_2 +
                                   (2 * a_3 - 25 * a_2 + 72 * a - 61) * c1 / 2 +
                                   (25 * a_3 - 195 * a_2 + 477 * a - 379) / 12)

                    y_2 = y * y
                    y_3 = y_2 * y
                    y_4 = y_2 * y_2
                    result = y + c1 + (c2 / y) + (c3 / y_2) + (c4 / y_3) + (c5 / y_4)
                else:
                    '''DiDonato and Morris Eq 33:'''
                    u = -lb + (a - 1) * np.log(w) - np.log(1 + (1 - a) / (1 + w))
                    result = -lb + (a - 1) * np.log(u) - np.log(1 + (1 - a) / (1 + u))
        else:
            z = w
            ap1 = a + 1
            ap2 = a + 2
            if (w < 0.15 * ap1):
                '''DiDonato and Morris Eq 35:'''
                v = lgamma_05
                z = np.exp((v + w) / a)
                s = np.log1p(z / ap1 * (1 + z / ap2))
                z = np.exp((v + z - s) / a)
                s = np.log1p(z / ap1 * (1 + z / ap2))
                z = np.exp((v + z - s) / a)
                s = np.log1p(z / ap1 * (1 + z / ap2 * (1 + z / (a + 3))))
                z = np.exp((v + z - s) / a)

            if ((z <= 0.01 * ap1) or (z > 0.7 * ap1)):
                result = z
            else:
                '''DiDonato and Morris Eq 36:'''
                ls = np.log(didonato_SN(a, z, 100, 1e-4))
                v = np.log(p) + lgamma_05
                z = np.exp((v + z - ls) / a)
                result = z * (1 - (a * np.log(z) - z - v + ls) / (a - z))
    return result

def igam_series(a, x):
    ax = igam_fac(a, x)
    if (ax == 0.0):
        return 0.0

    '''Power series'''
    r = a
    c = 1.0
    ans = 1.0

    for i in range(0, igam_MAXITER):
        r += 1.0
        c *= x / r
        ans += c
        if (c <= MACHEP * ans):
            break
    return (ans * ax / a)

def igam(a, x):
    if (a == 0):
        return 1 if (x > 0) else np.nan
    elif (x == 0):
        '''Zero integration limit'''
        return 0;
    elif np.isinf(a):
        return np.nan if np.isinf(x) else 0
    elif np.isinf(x):
        return 1
    if ((x > 1.0) and (x > a)):
        return (1.0 - _igamc(a, x))
    return igam_series(a, x)

def igami(a, p):
    if (np.isnan(a) or np.isnan(p)):
        return np.nan
    elif (p == 0.0):
        return 0.0
    elif (p == 1.0):
        return np.inf
    elif (p > 0.9):
        return igamci(a, 1 - p)

    x = find_inverse_gamma(a, p, 1 - p);
    '''Halley's method'''
    for i in range(0, 3):
        fac = igam_fac(a, x);
        if (fac == 0.0):
            return x
        f_fp = (igam(a, x) - p) * x / fac
        '''The ratio of the first and second derivatives simplifies'''
        fpp_fp = -1.0 + (a - 1) / x
        if (np.isinf(fpp_fp)):
            '''Resort to Newton's method in the case of overflow'''
            x = x - f_fp
        else:
            x = x - f_fp / (1.0 - 0.5 * f_fp * fpp_fp)
    return x

def igamci(a, q):
    if (q == 0.0):
        return np.inf
    elif (q == 1.0):
        return 0.0
    elif (q > 0.9):
        return igami(a, 1 - q)

    x = find_inverse_gamma(a, 1 - q, q)
    for i in range(0, 3):
        fac = igam_fac(a, x)
        if (fac == 0.0):
            return x
        f_fp = (_igamc(a, x) - q) * x / (-fac)
        fpp_fp = -1.0 + (a - 1) / x
        if np.isinf(fpp_fp):
            x = x - f_fp
        else:
            x = x - f_fp / (1.0 - 0.5 * f_fp * fpp_fp)
    return x

def chdtri(y, df):
    x = igamci(0.5 * df, y)
    return (2.0 * x)

In [19]:
math.lgamma(0.5)

0.5723649429247004

In [20]:
chdtri(0.0, df=1), chi2.isf(0.0, df=1)

(inf, inf)

In [23]:
def test(f, vals=np.linspace(0.01, 10, 1000)):
    for i in vals:
        f(i, 1)

In [21]:
for v in np.linspace(0.0, 1, 1000):
    print(abs(chi2.isf(v, df=1) - chdtri(v, df=1)))

C:\Users\vasiliev\AppData\Local\Temp\ipykernel_27488\1032258982.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(abs(chi2.isf(v, df=1) - chdtri(v, df=1)))


nan
6.572520305780927e-14
4.618527782440651e-14
5.1514348342607263e-14
5.1514348342607263e-14
1.0835776720341528e-13
8.171241461241152e-14
6.572520305780927e-14
1.0391687510491465e-13
7.904787935331115e-14
1.1013412404281553e-13
1.5987211554602254e-13
1.0391687510491465e-13
1.509903313490213e-13
1.0125233984581428e-13
1.3944401189291966e-13
1.8207657603852567e-13
1.2434497875801753e-13
1.545430450278218e-13
1.0658141036401503e-13
1.3145040611561853e-13
1.7141843500212417e-13
1.092459456231154e-13
1.3855583347321954e-13
1.794120407794253e-13
2.0961010704922955e-13
1.3855583347321954e-13
1.687538997430238e-13
2.078337502098293e-13
1.438849039914203e-13
1.7141843500212417e-13
1.9895196601282805e-13
1.438849039914203e-13
1.6786572132332367e-13
1.9628743075372768e-13
2.2470914018413168e-13
1.5276668818842154e-13
1.8207657603852567e-13
2.1671553440683056e-13
2.4780177909633494e-13
1.616484723854228e-13
1.9806378759312793e-13
2.2737367544323206e-13
2.646771690706373e-13
1.9184653865522705e-13

In [24]:
%timeit test(chi2.isf, vals=np.linspace(0.0, 1, 1000))

106 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%timeit test(chdtri, vals=np.linspace(0.0, 1, 1000))

14.3 ms ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
from scipy.stats import chi2

%timeit test(chi2.sf)

76.8 ms ± 71.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%timeit test(chisqprob_our)

336 µs ± 478 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [28]:
chi2.sf(19, 1) - chisqprob_our(19, 1)

-2.236166980751353e-19